In [1]:
# | default_exp crawler.crawler

In [2]:
#| exporti
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from bs4 import BeautifulSoup
# import time


In [3]:
#| export
def driversetup(is_headless: bool = True) -> webdriver:
    options = webdriver.ChromeOptions()
    # run Selenium in headless mode

    if is_headless:
        options.add_argument("--headless")

    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)

    return driver

In [4]:
driversetup()

<selenium.webdriver.chrome.webdriver.WebDriver (session="34cdbf9f1d88d57ef2fc4bfadf521ac9")>

In [5]:
#| export
def wait_and_return(
    driver: webdriver,
    element_id: str,
    el_type=By.ID,
    min_sleep_time=15,
    return_soup: bool = False,
):
    """
    Wait for the element to be present and return it.
    """

    try:
        res = WebDriverWait(driver, timeout=15, poll_frequency=1).until(
            EC.presence_of_element_located((el_type, element_id)))

        if return_soup:
            return BeautifulSoup(res.get_attribute("innerHTML"))
        return res

    except Exception as e:
        print(e)
        print(
            f"Timeout Exception: did not load within {min_sleep_time} seconds.")


In [11]:
#| export

def authenticate_driver(driver, domo_instance, domo_username, domo_password):

    url = f"https://{domo_instance}.domo.com/auth/index"

    driver.get(url)

    button = wait_and_return(driver, element_id="sign-in",
                             el_type=By.CLASS_NAME, min_sleep_time=4)

    form_username = wait_and_return(driver, "username", el_type=By.NAME)

    form_password = wait_and_return(driver, "password", el_type=By.NAME)

    # Sending input
    form_username.clear()
    form_username.send_keys(domo_username)
    form_password.clear()
    form_password.send_keys(domo_password)
    button.click()

    return driver

In [12]:
#| hide
import os
from dotenv import load_dotenv

load_dotenv()

domo_instance = os.environ["DOMO_INSTANCE"]
domo_username = os.environ["DOMO_USERNAME"]
domo_password = os.environ["DOMO_PASSWORD"]

In [13]:
driver = driversetup(is_headless=False)

driver = authenticate_driver(
    driver,
    domo_instance=domo_instance,
    domo_password=domo_password,
    domo_username=domo_username,
)